In [1]:
import sys
sys.path.append('..')

from estimators import *
from utils.model import Model
from utils.dataset import Dataset
from utils.processor import Logger
from utils.manager import UEManager
from utils.manager import estimate_uncertainty
from ue_metrics import ReversedPairsProportion, PredictionRejectionArea, RiskCoverageCurveAUC
from estimators import MaxProbabilityToken, LexicalSimilarity, SemanticEntropy, MutualInformationToken
from generation_metrics import RougeMetric, WERTokenwiseMetric, BartScoreSeqMetric, ModelScoreSeqMetric, ModelScoreTokenwiseMetric

/home/user/conda/envs/lm_poly/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-27 16:25:01.308901: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 16:25:02.494121: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model = Model.from_pretrained(
    'google/flan-t5-base',
    device='cuda:0',
)

In [3]:
dataset = Dataset.load(
    "../workdir/data/triviaqa.csv",
    'question', 'answer',
    batch_size=4,
)
dataset.subsample(200, seed=42)

In [4]:
ue_methods = [MaxProbabilitySeq(), SemanticEntropy(),]
ue_metrics = [RiskCoverageCurveAUC()]
metrics = [RougeMetric('rougeL')]
loggers = [Logger()] 

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
man = UEManager(
    dataset,
    model,
    ue_methods,
    metrics,
    ue_metrics,
    loggers,
)

In [6]:
results = man()

  0%|          | 0/50 [00:00<?, ?it/s]/home/user/conda/envs/lm_poly/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
  2%|▏         | 1/50 [00:18<14:51, 18.20s/it]

================================================== NEW BATCH ==================================================
Statistics:

input_texts: ['Which golf course has holes called Silver Tassie, Blink Bonnie and Wee Bogle?', "Which musical features the song 'Second Hand Rose'?", 'What is the name of the family’s pet goldfish in the US television cartoon series ‘American Dad’?', 'What red fruit-juice-based drink whose name alludes to energy was first produced by John Noel Nichols in Manchester, 1908?']

target_texts: ['GLENEAGLES', 'FUNNY GIRL', 'Klaus Heissler', 'Vimto']

target_tokens: [[3, 13011, 25576, 8418, 12335, 1, 1], [26280, 12056, 3, 7214, 12831, 1, 1], [17566, 216, 159, 7, 1171, 1, 1], [1813, 51, 235, 1, 1]]

generation_params: {}

ensemble_model: None

input_tokens: {'input_ids': [[4073, 4319, 503, 65, 8034, 718, 5642, 332, 6500, 15, 6, 272, 4907, 29203, 11, 101, 15, 24586, 109, 58, 1], [4073, 4183, 753, 8, 2324, 3, 31, 134, 15, 1018, 26, 2263, 5088, 31, 58, 1], [363, 19, 8, 564,

  4%|▍         | 2/50 [00:32<12:42, 15.90s/it]

================================================== NEW BATCH ==================================================
Statistics:

input_texts: ['What information technology company was founded by Sergey Brin and Larry Page?', 'Apart from a battle, what did Nelson lose at Tenerife in 1797?', 'Fireworks that burn red contain which element, the only one named after a place in UK?', 'What is the US state capital of Alabama?']

target_texts: ['Google', 'His arm', 'Strontium', 'Montgomery']

target_tokens: [[1163, 1, 1], [978, 2939, 1, 1], [472, 4438, 2552, 1, 1], [22948, 1, 1]]

generation_params: {}

ensemble_model: None

input_tokens: {'input_ids': [[363, 251, 748, 349, 47, 5710, 57, 20843, 63, 7834, 29, 11, 17129, 5545, 58, 1], [13323, 45, 3, 9, 3392, 6, 125, 410, 14448, 2615, 44, 4738, 49, 99, 15, 16, 1003, 4327, 58, 1], [3655, 13631, 24, 5958, 1131, 3480, 84, 3282, 6, 8, 163, 80, 2650, 227, 3, 9, 286, 16, 1270, 58, 1], [363, 19, 8, 837, 538, 1784, 13, 13050, 58, 1]], 'attention_mask': [[1, 

  6%|▌         | 3/50 [00:50<13:03, 16.68s/it]

...

================================================== NEW BATCH ==================================================
Statistics:

input_texts: ['"In which old children\'s TV programme did the ""biggest spotty dog you ever did see"" feature?"', 'One of the Wonders of the Modern World, The Catacombs of Alexandria are in which country?', 'In medicine, Epistaxiophobia is the fear of what?', 'The Babylonians invented the clepsydra. What was it?']

target_texts: ['THE WOODENTOPS', 'Egypt', 'Nosebleeds', 'Water clock']

target_tokens: [[1853, 3, 29637, 6431, 4652, 134, 1, 1], [10438, 1, 1], [465, 7, 15, 27779, 7, 1, 1], [2336, 6702, 1, 1]]

generation_params: {}

ensemble_model: None

input_tokens: {'input_ids': [[96, 1570, 84, 625, 502, 31, 7, 1424, 2486, 410, 8, 96, 121, 12911, 122, 222, 2140, 17, 63, 1782, 25, 664, 410, 217, 121, 121, 1451, 4609, 1], [555, 13, 8, 16347, 7, 13, 8, 5070, 1150, 6, 37, 3431, 9, 9763, 7, 13, 25664, 33, 16, 84, 684, 58, 1], [86, 4404, 6, 10395, 343, 8606, 32, 27426, 

 98%|█████████▊| 49/50 [16:51<00:23, 23.24s/it]

================================================== NEW BATCH ==================================================
Statistics:

input_texts: ["What was Jacques Cousteau's mobile oceonagraphic laboratory.", 'American singer Al, late British television host Hughie and British businessman Philip all share which surname?', 'What is the name of the first digital library founded by Michael Hart in 1971?', "In which decade did Coates compose 'The Dam Busters March'?"]

target_texts: ['Calypso', 'Green', 'Project Gutenberg', '1950s']

target_tokens: [[1336, 120, 102, 7, 32, 1, 1], [1862, 1, 1], [2786, 7756, 11063, 1, 1], [10247, 7, 1, 1]]

generation_params: {}

ensemble_model: None

input_tokens: {'input_ids': [[363, 47, 3, 17453, 11356, 849, 402, 31, 7, 1156, 3, 32, 565, 106, 9, 14797, 10343, 5, 1], [797, 7634, 901, 6, 1480, 2390, 4390, 2290, 3455, 5649, 15, 11, 2390, 268, 348, 12092, 66, 698, 84, 244, 4350, 58, 1], [363, 19, 8, 564, 13, 8, 166, 1125, 3595, 5710, 57, 2457, 10498, 16, 17961, 58,

100%|██████████| 50/50 [17:11<00:00, 20.63s/it]

================================================== NEW BATCH ==================================================
Statistics:

input_texts: ['"In what country, between the 1950s and 1980, would one have found ""Rusticated Youth""?"', 'What is the symbol for the chemical element protactinium, atomic number 91?', '"Which 19th century English writer wrote ""Swallows and Amazons"", ""Peter Duck"" and ""Secret Water""?"', 'Which country took over the Presidency of the European Union from Sweden on 1st. January 2010?']

target_texts: ['China', 'Pa', 'Arthur Ransome', 'SPAIN']

target_tokens: [[1473, 1, 1], [2709, 1, 1], [13962, 15039, 5529, 1, 1], [6760, 13570, 1, 1]]

generation_params: {}

ensemble_model: None

input_tokens: {'input_ids': [[96, 1570, 125, 684, 6, 344, 8, 10247, 7, 11, 6694, 6, 133, 80, 43, 435, 96, 121, 17137, 7, 1225, 920, 9452, 121, 121, 4609, 1], [363, 19, 8, 6083, 21, 8, 5368, 3282, 813, 17, 2708, 77, 2552, 6, 3, 20844, 381, 3, 4729, 58, 1], [96, 518, 107, 362, 957, 189,

In [7]:
results

{('sequence',
  'MaxProbabilitySeq',
  'Rouge_rougeL',
  'rcc-auc'): 0.9825768062848464,
 ('sequence', 'SemanticEntropy', 'Rouge_rougeL', 'rcc-auc'): 0.972887639048123}